In [1]:
!pip install scikit-learn

ERROR: Could not find a version that satisfies the requirement scikit-learn (from versions: none)
ERROR: No matching distribution found for scikit-learn


In [ ]:

# Set the dataset directory path (update if needed)
dataset_dir = r'C:/Users/user/Desktop/hand_gesture/ultralytics-cpu/ultralytics/data/dataset'

image_count = 0
matched_count = 0

for f in os.listdir(dataset_dir):
    if f.endswith(('.jpg', '.png')):
        image_count += 1
        txt = os.path.splitext(f)[0] + '.txt'
        if os.path.exists(os.path.join(dataset_dir, txt)):
            matched_count += 1
        else:
            print(f"❌ Missing label for: {f}")

print(f"\n📊 Total image files: {image_count}")
print(f"✅ Matched image-label pairs: {matched_count}")
print(f"⚠️ Unmatched files: {image_count - matched_count}")


📊 Total image files: 651
✅ Matched image-label pairs: 651
⚠️ Unmatched files: 0


In [ ]:

# Path to your dataset where all images and labels are stored
dataset_dir = r'C:/Users/user/Desktop/hand_gesture/ultralytics-cpu/ultralytics/data/dataset'

# Output folders
output_images = os.path.join(dataset_dir, 'images')
output_labels = os.path.join(dataset_dir, 'labels')

# Create train/val/test subfolders
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_images, split), exist_ok=True)
    os.makedirs(os.path.join(output_labels, split), exist_ok=True)

# Match image-label pairs
image_label_pairs = []
for file in os.listdir(dataset_dir):
    if file.endswith(('.jpg', '.png')):
        label = os.path.splitext(file)[0] + '.txt'
        if os.path.exists(os.path.join(dataset_dir, label)):
            image_label_pairs.append((file, label))

# Split into train, val, test
train_pairs, temp_pairs = train_test_split(image_label_pairs, test_size=0.2, random_state=42)
val_pairs, test_pairs = train_test_split(temp_pairs, test_size=0.5, random_state=42)

# Function to copy files
def copy_files(pairs, split):
    for img_file, lbl_file in pairs:
        shutil.copy(os.path.join(dataset_dir, img_file), os.path.join(output_images, split, img_file))
        shutil.copy(os.path.join(dataset_dir, lbl_file), os.path.join(output_labels, split, lbl_file))

# Copy the files
copy_files(train_pairs, 'train')
copy_files(val_pairs, 'val')
copy_files(test_pairs, 'test')

print("✅ Dataset successfully split into train/val/test folders!")

✅ Dataset successfully split into train/val/test folders!


In [ ]:
from pathlib import Path

labels_path = Path("C:/Users/user/Desktop/hand_gesture/ultralytics-cpu/ultralytics/data/dataset/labels/train")

for txt in labels_path.glob("*.txt"):
    content = txt.read_text().strip()
    if not content:
        print(f"[EMPTY FILE] {txt}")
    else:
        for line in content.splitlines():
            parts = line.split()
            if len(parts) != 5:
                print(f"[BAD FORMAT] {txt}: {line}")


In [ ]:
from pathlib import Path

LABEL_DIR = Path("C:/Users/user/Desktop/hand_gesture/ultralytics-cpu/ultralytics/data/dataset/labels/train")

print("📂 Checking label files...\n")
for txt in LABEL_DIR.glob("*.txt"):
    content = txt.read_text().strip()
    if not content:
        print(f"❌ EMPTY LABEL FILE: {txt.name}")
    else:
        for i, line in enumerate(content.splitlines()):
            parts = line.split()
            if len(parts) != 5:
                print(f"⚠️ BAD FORMAT in {txt.name} (line {i+1}): '{line}'")
            else:
                try:
                    cls = int(parts[0])
                    if cls < 0 or cls > 9:
                        print(f"🚫 INVALID CLASS INDEX in {txt.name} (line {i+1}): {cls}")
                except ValueError:
                    print(f"❌ INVALID VALUE in {txt.name} (line {i+1}): '{line}'")


In [ ]:
from pathlib import Path
from PIL import Image

# Change these paths to match your actual dataset
image_dir = Path("C:/Users/user/Desktop/hand_gesture/ultralytics-cpu/ultralytics/data/dataset/images/train")
label_dir = Path("C:/Users/user/Desktop/hand_gesture/ultralytics-cpu/ultralytics/data/dataset/labels/train")

print("🔍 Scanning image–label consistency...\n")

bad_files = 0

for img_path in image_dir.glob("*.jpg"):  # or "*.png" depending on your image format
    label_path = label_dir / (img_path.stem + ".txt")
    
    if not label_path.exists():
        print(f"❌ Missing label for image: {img_path.name}")
        bad_files += 1
        continue

    try:
        with Image.open(img_path) as im:
            im.verify()
    except Exception as e:
        print(f"🛑 Corrupt image: {img_path.name} — {e}")
        bad_files += 1

    content = label_path.read_text().strip()
    if not content:
        print(f"⚠️ Empty label file: {label_path.name}")
        bad_files += 1
    else:
        for i, line in enumerate(content.splitlines()):
            parts = line.split()
            if len(parts) != 5:
                print(f"❗ Bad format in {label_path.name} (line {i+1}): {line}")
                bad_files += 1
            else:
                try:
                    cls = int(parts[0])
                    if cls < 0 or cls > 25:
                        print(f"🚫 Invalid class index in {label_path.name} (line {i+1}): {cls}")
                        bad_files += 1
                except ValueError:
                    print(f"❌ Non-integer class in {label_path.name} (line {i+1}): {line}")
                    bad_files += 1

print(f"\n✅ Check complete. {bad_files} potential issues found.")


In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load trained model
model_path = r'C:/Users/user/Desktop/hand_gesture/output/nsl_yolo_train2/weights/best.pt'
model = YOLO(model_path)

# Input image path
image_path = r"C:\Users\user\Desktop\hand_gesture\ultralytics-cpu\ultralytics\data\dataset\images\test\A_1752579012993.jpg"

# Run prediction
results = model.predict(source=image_path, imgsz=320, conf=0.25, save=True)

# Get path to the saved result
result_dir = results[0].save_dir  # Example: runs\detect\predict5
result_file = os.path.join(result_dir, os.path.basename(image_path))

# Load and display the image using OpenCV
img = cv2.imread(result_file)
cv2.imshow("Predicted Output", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("✅ Prediction complete! Displayed result in a window.")


In [ ]:
from ultralytics import YOLO
import cv2
import os
import glob

# Load trained model
# model_path = r"C:\Users\USER\Desktop\Fx_Work\hand_gesture\output\nsl_yolo_train2\weights\best.pt"
model = YOLO("best.pt")

# Input image path
image_path = r"C:\Users\USER\Desktop\Fx_Work\hand_gesture\ultralytics-cpu\ultralytics\data\dataset\images\test\A_1752579012993.jpg"

# Run prediction
results = model.predict(source=image_path, imgsz=320, conf=0.25, save=True)

# Automatically get path to the last saved result
result_dir = results[0].save_dir  # e.g., runs/detect/predict5
# Find the most recent image saved in the result directory
saved_images = glob.glob(os.path.join(result_dir, "*.jpg"))
if not saved_images:
    saved_images = glob.glob(os.path.join(result_dir, "*.png"))

if saved_images:
    result_file = max(saved_images, key=os.path.getctime)  # latest file
else:
    raise FileNotFoundError("No output image found in YOLO result directory.")

# Load and display the image using OpenCV
img = cv2.imread(result_file)
cv2.imshow("Predicted Output", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(f"✅ Prediction complete! Displayed result: {result_file}")


🚀 Loaded model: best.pt
Invalid option selected


In [9]:
from ultralytics import YOLO
import cv2
import os
import glob
import time
from pathlib import Path

def clear_previous_runs():
    """Clear previous YOLO prediction directories"""
    runs_detect = Path("runs/detect")
    if runs_detect.exists():
        for item in runs_detect.glob("predict*"):
            try:
                if item.is_dir():
                    for file in item.glob("*"):
                        file.unlink()
                    item.rmdir()
            except Exception as e:
                print(f"⚠️ Could not delete {item}: {e}")

def test_model(model_path, image_path, conf_threshold=0.25, imgsz=320):
    """
    Test YOLOv8 model on a single image with enhanced features
    
    Args:
        model_path (str): Path to the trained model (.pt file)
        image_path (str): Path to the test image
        conf_threshold (float): Confidence threshold (0-1)
        imgsz (int): Image size for inference
    """
    # Validate inputs
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    print(f"🔍 Testing model: {os.path.basename(model_path)}")
    print(f"📷 Input image: {os.path.basename(image_path)}")
    
    # Clear previous runs to avoid confusion
    clear_previous_runs()
    
    # Load model with error handling
    try:
        model = YOLO(model_path)
        print("✅ Model loaded successfully")
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")
    
    # Record inference time
    start_time = time.time()
    
    # Run prediction with additional parameters
    results = model.predict(
        source=image_path,
        imgsz=imgsz,
        conf=conf_threshold,
        save=True,
        save_txt=True,  # Save labels to text file
        save_conf=True,  # Save confidence scores in text file
        exist_ok=True    # Overwrite existing files
    )
    
    inference_time = time.time() - start_time
    print(f"⏱️ Inference time: {inference_time:.2f} seconds")
    
    # Get result directory and files
    result_dir = results[0].save_dir
    print(f"📂 Results saved to: {result_dir}")
    
    # Find the output image (supports multiple formats)
    image_formats = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    saved_images = []
    for fmt in image_formats:
        saved_images.extend(glob.glob(os.path.join(result_dir, fmt)))
    
    if not saved_images:
        raise FileNotFoundError("No output image found in result directory")
    
    # Get most recent image (handles multiple detections)
    result_file = max(saved_images, key=os.path.getctime)
    
    # Display results with OpenCV
    try:
        img = cv2.imread(result_file)
        if img is None:
            raise ValueError("Could not read the result image")
        
        # Resize for display if too large
        max_display_size = 1200
        h, w = img.shape[:2]
        if max(h, w) > max_display_size:
            scale = max_display_size / max(h, w)
            img = cv2.resize(img, (int(w*scale), int(h*scale)))
        
        # Create named window with proper size
        cv2.namedWindow("Predicted Output", cv2.WINDOW_NORMAL)
        cv2.imshow("Predicted Output", img)
        
        # Add key instructions
        print("\nℹ️ Press any key to close the window or 's' to save the result")
        
        key = cv2.waitKey(0)
        if key == ord('s'):
            save_path = os.path.join(os.path.dirname(image_path), "saved_result.jpg")
            cv2.imwrite(save_path, img)
            print(f"💾 Result saved to: {save_path}")
        
        cv2.destroyAllWindows()
        
        # Print detection details
        print("\n📊 Detection Results:")
        for i, result in enumerate(results):
            print(f"  - Image {i+1}:")
            if len(result.boxes) == 0:
                print("    No detections found")
            else:
                for box, conf, cls_id in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
                    cls_name = result.names[int(cls_id)]
                    print(f"    Detected: {cls_name} (confidence: {conf:.2f}, box: {box.tolist()})")
        
        print(f"\n✅ Prediction complete! Result displayed: {result_file}")
        
    except Exception as e:
        print(f"⚠️ Error displaying results: {e}")
        raise

if __name__ == "__main__":
    # Configuration - update these paths
    MODEL_PATH = "best.pt"
    IMAGE_PATH = r"C:\Users\USER\Desktop\Fx_Work\hand_gesture\ultralytics-cpu\ultralytics\data\dataset\images\test\A_1752579012993.jpg"
    
    # Run the test
    try:
        test_model(MODEL_PATH, IMAGE_PATH)
    except Exception as e:
        print(f"❌ Error: {e}")

🔍 Testing model: best.pt
📷 Input image: A_1752579012993.jpg
✅ Model loaded successfully

image 1/1 C:\Users\USER\Desktop\Fx_Work\hand_gesture\ultralytics-cpu\ultralytics\data\dataset\images\test\A_1752579012993.jpg: 320x320 3 As, 70.2ms
Speed: 3.1ms preprocess, 70.2ms inference, 2.4ms postprocess per image at shape (1, 3, 320, 320)
Results saved to runs\detect\predict
1 label saved to runs\detect\predict\labels
⏱️ Inference time: 2.87 seconds
📂 Results saved to: runs\detect\predict

ℹ️ Press any key to close the window or 's' to save the result

📊 Detection Results:
  - Image 1:
    Detected: A (confidence: 1.00, box: [31.14107322692871, 19.55372428894043, 179.57029724121094, 196.63958740234375])
    Detected: A (confidence: 1.00, box: [33.08327865600586, 17.897254943847656, 129.7235870361328, 209.98028564453125])
    Detected: A (confidence: 1.00, box: [33.34016418457031, 24.127395629882812, 197.5474090576172, 238.20034790039062])

✅ Prediction complete! Result displayed: runs\detect\